In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
import os
import numpy as np
from shapely.geometry import box
from tqdm.notebook import tqdm

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

D:\ANACONDA3\envs\py38-env\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.11.0-CAPI-1.17.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
os.listdir('./원본데이터/격자/')

['200x200_격자', '250x250_격자', '포인트']

In [3]:
def grid_250(file_name, epsg, encoding='cp949'):
    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")

    # 김해시 그리드
    gimhae_grid = gpd.GeoDataFrame.from_file('./원본데이터/격자/250x250_격자/250x250_격자.shp', encoding=encoding)
    gimhae_grid.crs = epsg5179

    # 그리드와 병합할 포인트
    
    User_point = pd.read_csv(f'./전처리/grid/{file_name}.csv', encoding=encoding)
    
    User_point['x'] = None
    User_point['y'] = None

    for n in tqdm(range(len(User_point))):
        xy = User_point['geom'][n].split()
        User_point['x'][n] = float(xy[1].split('(')[1])
        User_point['y'][n] = float(xy[2].split(')')[0])
    
    User_point.drop('geom', axis=1, inplace=True)
    User_point['geometry'] = User_point.apply(lambda row : Point([row['x'], row['y']]), axis=1)
    
    gimhae_grid_point = gpd.GeoDataFrame(User_point, geometry='geometry', crs = epsg5179)

    if epsg == 4326:
        gimhae_grid_point.crs = epsg4326
    elif epsg == 5179:
        gimhae_grid_point.crs = epsg4326
        gimhae_grid_point.crs = epsg5179

    # 공간 병합
    result = gpd.sjoin(gimhae_grid, gimhae_grid_point, how='left', op="intersects")

    # 값이 없는 그리드 삭제
    result = result[result['sum_pop']>0]
    
    # 1개의 격자로 sum
    result = result.groupby(['id','left', 'top', 'right', 'bottom', 'year'])['sum_pop', 'h_pop', 'w_pop', 'v_pop'].sum()
    result.reset_index(inplace=True)
    
    # geometry 컬럼 생성
    result['geometry'] = result.apply(lambda row : 
                                  Polygon([(row['left'], row['top']), 
                                           (row['right'], row['top']),
                                           (row['right'], row['bottom']), 
                                           (row['left'], row['bottom'])]), axis=1)
    
    # GeoPandas로 변경
    result = gpd.GeoDataFrame(result, geometry='geometry', crs = epsg5179)
    
    # 폴더 생성
    try:
        directory = f'./전처리/grid/250x250/{file_name}/'
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
        
    # shp파일 저장
    result.to_file(f'{directory}{file_name}.shp', driver='ESRI Shapefile')

In [4]:
grid_250('agg_pcell_time_service_pop_2022_1~12', 5179)

  0%|          | 0/82749 [00:00<?, ?it/s]

In [5]:
epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")

# 김해시 그리드
gimhae_grid = gpd.GeoDataFrame.from_file('./전처리/grid/250x250/agg_pcell_time_service_pop_2022_1~8/agg_pcell_time_service_pop_2022_1~8.shp', encoding='cp949')
gimhae_grid.crs = epsg5179

# 그리드와 병합할 포인트

gimhae_grid_point = gpd.GeoDataFrame.from_file(f'D:/김해시청 프로젝트/SKT_유동인구/김해시_관광지별_유동인구/관광지/관광지.shp', encoding='cp949', crs= epsg4326)
gimhae_grid_point.to_crs(epsg=5179, inplace=True)

In [6]:
result = gpd.sjoin(gimhae_grid_point, gimhae_grid, how='left', op="intersects")

In [7]:
result

관광지명           X          Y  \
0               가야 컨트리클럽  128.896582  35.273991   
1                   가야랜드  128.902010  35.260511   
2                  가야의거리  128.872355  35.235574   
3            국립김해박물관_구지봉  128.875814  35.243011   
4               글로벌 푸드타운  128.882145  35.234849   
5               김해가야테마파크  128.893064  35.250597   
6              김해낙동강레일파크  128.820828  35.371798   
7               김해롯데워터파크  128.829305  35.180878   
8              김해목재문화박물관  128.808133  35.180060   
9                김해문화의전당  128.869385  35.243882   
10                김해상록GC  128.797531  35.272330   
11                 김해천문대  128.887301  35.253119   
12               김해한옥체험관  128.876976  35.234133   
13              내외동 먹자골목  128.866690  35.233189   
14                대동승마랜드  128.962447  35.248789   
15              대성동고분박물관  128.873092  35.236471   
16             봉리단길_봉황대길  128.880156  35.230334   
17         봉하마을_노무현대통령생가  128.771213  35.314445   
18                 부경동물원  128.819867  35.220315   
19                서부문화센터  128.820945  35.172604   
20                  수로왕릉  128.878496  35.235310   
21         수로왕비릉_청소년문화의집  128.878400  35.242038   
22           수릉원_김해민속박물관  128.875724  35.235508   
23          신어산 산림욕장_은하사  128.912593  35.265758   
24                  연지공원  128.868820  35.246382   
25              장유사_대청계곡  128.775948  35.185053   
26           장유스파랜드_장유온천  128.806993  35.187889   
27                  장유폭포  128.771908  35.187639   
28               정산컨트리클럽  128.805608  35.255237   
29  클레이아크김해미술관_김해분청도자박물관  128.745862  35.251542   
30            화포천습지 생태공원  128.788794  35.310978   

                           geometry  index_right       id          left  \
0   POINT (1127215.769 1698496.445)         4863  11327.0  1.127118e+06   
1   POINT (1127533.208 1697019.179)         4927  11448.0  1.127368e+06   
2   POINT (1124892.735 1694053.099)         4281  10310.0  1.124868e+06   
3   POINT (1125177.058 1695044.955)         4359  10421.0  1.125118e+06   
4   POINT (1125765.709 1694147.684)         4518  10654.0  1.125618e+06   
5   POINT (1126734.846 1695908.222)         4759  11107.0  1.126618e+06   
6   POINT (1119983.595 1709260.953)         2501   7949.0  1.119868e+06   
7   POINT (1121008.600 1688149.324)         2923   8493.0  1.120868e+06   
8   POINT (1119110.711 1687979.799)         2238   7574.0  1.118868e+06   
9   POINT (1124594.857 1695099.120)         4116  10076.0  1.124368e+06   
10  POINT (1118004.308 1698088.931)         1872   7074.0  1.117868e+06   
11  POINT (1126206.610 1696180.535)         4639  10876.0  1.126118e+06   
12  POINT (1125296.436 1694061.698)         4363  10425.0  1.125118e+06   
13  POINT (1124361.863 1693944.072)         4031   9965.0  1.124118e+06   
14  POINT (1133050.366 1695798.439)         5886  13983.0  1.132868e+06   
15  POINT (1124939.416 1694316.156)         4280  10309.0  1.124868e+06   
16  POINT (1125591.671 1693644.441)         4443  10541.0  1.125368e+06   
17  POINT (1115557.962 1702840.816)         1106   5905.0  1.115368e+06   
18  POINT (1120119.961 1692458.585)         2638   8131.0  1.120118e+06   
19  POINT (1120288.376 1687168.316)         2659   8152.0  1.120118e+06   
20  POINT (1125432.926 1694194.136)         4441  10539.0  1.125368e+06   
21  POINT (1125413.903 1694940.219)         4438  10536.0  1.125368e+06   
22  POINT (1125180.365 1694212.599)         4362  10424.0  1.125118e+06   
23  POINT (1128484.508 1697682.797)         5148  11905.0  1.128368e+06   
24  POINT (1124564.518 1695344.593)         4115  10075.0  1.124368e+06   
25  POINT (1116172.901 1688495.506)         1381   6307.0  1.116118e+06   
26  POINT (1118995.534 1688846.705)         2235   7571.0  1.118868e+06   
27  POINT (1115801.330 1688777.623)         1230   6076.0  1.115618e+06   
28  POINT (1118923.454 1696291.321)         2206   7541.0  1.118868e+06   
29  POINT (1113341.015 1695834.994)          520   4898.0  1.113118e+06   
30  POINT (1117161.258 1702476.919)         1628   6711.0  1.1

In [8]:
result.columns

Index(['관광지명', 'X', 'Y', 'geometry', 'index_right', 'id', 'left', 'top',
       'right', 'bottom', 'year', 'sum_pop', 'h_pop', 'w_pop', 'v_pop'],
      dtype='object')

In [9]:
result = result[['관광지명', 'X', 'Y','sum_pop']]

In [10]:
result = result.astype({'sum_pop':int})

In [11]:
result['month_mean'] = round(result['sum_pop'] / 12)

In [12]:
result = result.astype({'month_mean':int})

In [13]:
result.to_excel('./전처리/관광지/정리/관광지별_유동인구_1~12월.xlsx')

In [7]:
result = result.groupby(['id','left', 'top', 'right', 'bottom', 'year', 'month', 'hcode'])['s_pop', 'h_pop', 'w_pop', 'v_pop'].sum()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2008\1048648293.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = result.groupby(['id','left', 'top', 'right', 'bottom', 'year', 'month', 'hcode'])['s_pop', 'h_pop', 'w_pop', 'v_pop'].sum()


In [8]:
result.reset_index(inplace=True)

In [9]:
result

id          left           top         right        bottom    year  \
0      2688.0  1.108368e+06  1.702122e+06  1.108618e+06  1.701872e+06  2022.0   
1      2689.0  1.108368e+06  1.701872e+06  1.108618e+06  1.701622e+06  2022.0   
2      2802.0  1.108618e+06  1.702372e+06  1.108868e+06  1.702122e+06  2022.0   
3      2804.0  1.108618e+06  1.701872e+06  1.108868e+06  1.701622e+06  2022.0   
4      2805.0  1.108618e+06  1.701622e+06  1.108868e+06  1.701372e+06  2022.0   
...       ...           ...           ...           ...           ...     ...   
6566  16045.0  1.137368e+06  1.697872e+06  1.137618e+06  1.697622e+06  2022.0   
6567  16047.0  1.137368e+06  1.697372e+06  1.137618e+06  1.697122e+06  2022.0   
6568  16157.0  1.137618e+06  1.698622e+06  1.137868e+06  1.698372e+06  2022.0   
6569  16158.0  1.137618e+06  1.698372e+06  1.137868e+06  1.698122e+06  2022.0   
6570  16159.0  1.137618e+06  1.698122e+06  1.137868e+06  1.697872e+06  2022.0   

      month         hcode   s_pop  h_pop  w_pop  v_pop  
0       1.0  4.825025e+09    0.11   0.11   0.02   0.04  
1       1.0  4.825025e+09    0.21   0.21   0.05   0.06  
2       1.0  4.825025e+09    0.02   0.02   0.00   0.00  
3       1.0  4.825025e+09    0.01   0.01   0.00   0.00  
4       1.0  4.825025e+09    0.02   0.02   0.00   0.00  
...     ...           ...     ...    ...    ...    ...  
6566    1.0  4.825037e+09   14.84   6.53   2.44   9.24  
6567    1.0  4.825037e+09    0.73   0.32   0.12   0.45  
6568    1.0  4.825037e+09  113.58  49.89  18.64  70.75  
6569    1.0  4.825037e+09    2.21   0.97   0.36   1.38  
6570    1.0  4.825037e+09    6.15   2.70   1.01   3.84  

[6571 rows x 12 columns]

In [10]:
result['geometry'] = result.apply(lambda row : 
                                  Polygon([(row['left'], row['top']), 
                                           (row['right'], row['top']),
                                           (row['right'], row['bottom']), 
                                           (row['left'], row['bottom'])]), axis=1)

In [11]:
result

id          left           top         right        bottom    year  \
0      2688.0  1.108368e+06  1.702122e+06  1.108618e+06  1.701872e+06  2022.0   
1      2689.0  1.108368e+06  1.701872e+06  1.108618e+06  1.701622e+06  2022.0   
2      2802.0  1.108618e+06  1.702372e+06  1.108868e+06  1.702122e+06  2022.0   
3      2804.0  1.108618e+06  1.701872e+06  1.108868e+06  1.701622e+06  2022.0   
4      2805.0  1.108618e+06  1.701622e+06  1.108868e+06  1.701372e+06  2022.0   
...       ...           ...           ...           ...           ...     ...   
6566  16045.0  1.137368e+06  1.697872e+06  1.137618e+06  1.697622e+06  2022.0   
6567  16047.0  1.137368e+06  1.697372e+06  1.137618e+06  1.697122e+06  2022.0   
6568  16157.0  1.137618e+06  1.698622e+06  1.137868e+06  1.698372e+06  2022.0   
6569  16158.0  1.137618e+06  1.698372e+06  1.137868e+06  1.698122e+06  2022.0   
6570  16159.0  1.137618e+06  1.698122e+06  1.137868e+06  1.697872e+06  2022.0   

      month         hcode   s_pop  h_pop  w_pop  v_pop  \
0       1.0  4.825025e+09    0.11   0.11   0.02   0.04   
1       1.0  4.825025e+09    0.21   0.21   0.05   0.06   
2       1.0  4.825025e+09    0.02   0.02   0.00   0.00   
3       1.0  4.825025e+09    0.01   0.01   0.00   0.00   
4       1.0  4.825025e+09    0.02   0.02   0.00   0.00   
...     ...           ...     ...    ...    ...    ...   
6566    1.0  4.825037e+09   14.84   6.53   2.44   9.24   
6567    1.0  4.825037e+09    0.73   0.32   0.12   0.45   
6568    1.0  4.825037e+09  113.58  49.89  18.64  70.75   
6569    1.0  4.825037e+09    2.21   0.97   0.36   1.38   
6570    1.0  4.825037e+09    6.15   2.70   1.01   3.84   

                                               geometry  
0     POLYGON ((1108368.1257 1702122.1241, 1108618.1...  
1     POLYGON ((1108368.1257 1701872.1241, 1108618.1...  
2     POLYGON ((1108618.1257 1702372.1241, 1108868.1...  
3     POLYGON ((1108618.1257 1701872.1241, 1108868.1...  
4     POLYGON ((1108618.1257 1701622.1241, 1108868.1...  
...                                                 ...  
6566  POLYGON ((1137368.1257 1697872.1241, 1137618.1...  
6567  POLYGON ((1137368.1257 1697372.1241, 1137618.1...  
6568  POLYGON ((1137618.1257 1698622.1241, 1137868.1...  
6569  POLYGON ((1137618.1257 1698372.1241, 1137868.1...  
6570  POLYGON ((1137618.1257 1698122.1241, 1137868.1...  

[6571 rows x 13 columns]

In [12]:
epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")
result = gpd.GeoDataFrame(result, geometry='geometry', crs = epsg5179)

In [13]:
# 폴더 생성
try:
    directory = f'./전처리/grid/250x250/{file_name}/'
    if not os.path.exists(directory):
        os.makedirs(directory)
except OSError:
    print ('Error: Creating directory. ' +  directory)


# shp파일 저장
result.to_file(f'{directory}{file_name}.shp', driver='ESRI Shapefile')